**Wav2Vec 2.0**

Pretrained model released by Facebook AI. (open-sourced)


Code adapted from https://www.analyticsvidhya.com/blog/2021/09/ok-google-speech-to-text-in-python-with-deep-learning-in-2-minutes/

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 37.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 59 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 39.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from pathlib import Path


import torch
import librosa
import numpy as np
import soundfile as sf
from scipy.io import wavfile
from IPython.display import Audio

from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
audio_data = '/content/drive/MyDrive/Master_courses/VR/DATA/'


In [19]:
#path to all .wav files
paths = list(Path(audio_data).rglob('*.wav'))
files = [x for x in paths if x.is_file()]

In [6]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:423: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,


Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
files

[PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0001 Hybrid 0.wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0001 Hybrid 3.wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0001 Hybrid 1.wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0001 Hybrid 4.wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0001 Hybrid 1 7.wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0002 FirstPerson 4 (mp3cut.net).wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0002 FirstPerson 4 (mp3cut.net) (1).wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/1637202023_1981485520output.wav'),
 PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/0002 FirstPerson 1 (mp3cut.net)_good_quality.wav')]

In [16]:
file_name = files[7]
file_name


PosixPath('/content/drive/MyDrive/Master_courses/VR/DATA/1637202023_1981485520output.wav')

In [9]:
data = wavfile.read(file_name)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sampling rate:',framerate,'Hz')

Sampling rate: 44100 Hz


**Resampling**

We need to resemple audio to 16000Hz, as Wav2Vec was pre-trained on 16 kHz frequency.  

In [10]:
def transcribe(file_name):
  input_audio, _ = librosa.load(file_name, sr=16000)
  input_values = tokenizer(input_audio, return_tensors="pt").input_values
  logits = model(input_values).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  transcription = tokenizer.batch_decode(predicted_ids)[0]
  print(transcription)

**Noisy file:**

*   experimenter and participant talke during the recording
*   some parts of the audios are just noise, i.e  silent moments (paticipant does not talk)


In [17]:
transcribe(files[7])

IS THIT RECORDING AYOR COULD A CAR I RECORD YOU RECORDING IN PROGRESS SO FOR THIS TASK I HAD TO A FOLLOW ISTEAD OF ARROWS THROUGH TE PAST UM AND THEN I JUST WOK AROUND EN THE WASIT THEN OK


Transcription wiht little preprocessing.

In [18]:
transcribe(files[6])

O CASE SO YE I GAIN SOME EXPERIENCE WITH NAVIGATION IS EASIER THE FIRST PERSON AND NOW I DO FEEL IT IT OL BE DIZZY NOT MUCH BUT JUST A LITTLE BIT AH I GUESS MAINLY BECAUSE OF THE ELERV


In [21]:
transcribe(files[8])

AH SI THIS TIME I WAS PLAIN OR IWAS IN THE INVIRONMENT FROM THE FIRST PERSONA PERSPECTIVE IT IS EASIER TO NAVIGATE THE INVIRONMENT AH GIVING THE PIRST PERSON PERSPECTIVE BUT I THINK I YOURE MORE LIKELY TO GET DIZZY AH FROM DISPESPECTIVE
